In [3]:
import pandas as pd

In [43]:
# Load and visualize the main informations of the databases

df_observations = pd.read_csv("observations.csv")

print(f"Observations database shape: {df_observations.shape}")
df_observations.head()

Observations database shape: (23296, 3)


,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [44]:
df_species_info = pd.read_csv("species_info.csv")

print(f"Species Info database shape: {df_species_info.shape}")
df_species_info.head()

Species Info database shape: (5824, 4)


,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN


In [45]:
# Checking for duplicates in species_info database

duplicates = df_species_info.duplicated()
print(df_species_info[duplicates])

Empty DataFrame
Columns: [category, scientific_name, common_names, conservation_status]
Index: []


In [46]:
# Count the number of species
unique_species = df_species_info['scientific_name'].nunique()
print(f"Number of unique species: {unique_species}")

Number of unique species: 5541


In [47]:
# Identify why are there duplicated scientific names

duplicated_sci_names = df_species_info.duplicated(subset=['scientific_name'], keep=False)
df_species_info[duplicated_sci_names].sort_values(['scientific_name'])

,category,scientific_name,common_names,conservation_status
5553,Vascular Plant,Agrostis capillaris,"Colonial Bent, Colonial Bentgrass",NaN
2132,Vascular Plant,Agrostis capillaris,Rhode Island Bent,NaN
2134,Vascular Plant,Agrostis gigantea,Redtop,NaN
5554,Vascular Plant,Agrostis gigantea,"Black Bent, Redtop, Water Bentgrass",NaN
4178,Vascular Plant,Agrostis mertensii,"Arctic Bentgrass, Northern Bentgrass",NaN
...,...,...,...,...
5643,Vascular Plant,Vulpia myuros,"Foxtail Fescue, Rattail Fescue, Rat-Tail Fescu...",NaN
2331,Vascular Plant,Vulpia octoflora,Annual Fescue,NaN
4290,Vascular Plant,Vulpia octoflora,"Eight-Flower Six-Weeks Grass, Pullout Grass, S...",NaN
3347,Vascular Plant,Zizia aptera,"Heartleaf Alexanders, Heart-Leaf Alexanders, M...",NaN


In [48]:
# Checar se as colunas category e conservation_status estão consistentes para as espécies duplicadas

check_consistency = df_species_info.groupby('scientific_name').agg({column: 'nunique' for column in df_species_info.columns if column != 'common_names'})

# Filtra as colunas e grupos onde o número de valores únicos é maior que 1
inconsistency = check_consistency.loc[(check_consistency > 1).any(axis=1)]

# Verifica se há inconsistências
if not inconsistency.empty:
    print("Inconsistências encontradas nas seguintes entradas:")
    print(inconsistency)
else:
    print("Todas as colunas são consistentes entre duplicatas do nome científico.")


Inconsistências encontradas nas seguintes entradas:
                 category  scientific_name  conservation_status
scientific_name                                                
Canis lupus             1                1                    2


In [37]:
df_species_info[df_species_info['scientific_name'] == 'Canis lupus']

,category,scientific_name,common_names,conservation_status
8,Mammal,Canis lupus,Gray Wolf,Endangered
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered


In [50]:
# Uniformizar o status de conservação da espécie Canis lupus
df_species_info.at[3020, 'conservation_status'] = 'Endangered'
df_species_info

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN
...,...,...,...,...
5819,Vascular Plant,Solanum parishii,Parish's Nightshade,NaN
5820,Vascular Plant,Solanum xanti,"Chaparral Nightshade, Purple Nightshade",NaN
5821,Vascular Plant,Parthenocissus vitacea,"Thicket Creeper, Virginia Creeper, Woodbine",NaN
5822,Vascular Plant,Vitis californica,"California Grape, California Wild Grape",NaN


In [51]:
# Unificar as espécies duplicadas, unindo os diferentes nomes comuns
df_species_info_group = df_species_info.groupby('scientific_name').agg({'common_names': lambda x: ', '.join(x.unique()),**{column: 'first' for column in df_species_info.columns if column != 'common_names' and column !='scientific_name'}}).reset_index()
df_species_info_group

,scientific_name,common_names,category,conservation_status
0,Abies bifolia,Rocky Mountain Alpine Fir,Vascular Plant,None
1,Abies concolor,"Balsam Fir, Colorado Fir, Concolor Fir, Silver...",Vascular Plant,None
2,Abies fraseri,Fraser Fir,Vascular Plant,Species of Concern
3,Abietinella abietina,Abietinella Moss,Nonvascular Plant,None
4,Abronia ammophila,"Wyoming Sand Verbena, Yellowstone Sand Verbena",Vascular Plant,Species of Concern
...,...,...,...,...
5536,Zonotrichia leucophrys gambelii,Gambel's White-Crowned Sparrow,Bird,None
5537,Zonotrichia leucophrys oriantha,Mountain White-Crowned Sparrow,Bird,None
5538,Zonotrichia querula,Harris's Sparrow,Bird,None
5539,Zygodon viridissimus,Zygodon Moss,Nonvascular Plant,None


In [52]:
# Count number of species by conservation_status
species_count_by_status = df_species_info_group.groupby('conservation_status')['scientific_name'].nunique()
species_count_by_status

conservation_status
Endangered             15
In Recovery             3
Species of Concern    151
Threatened             10
Name: scientific_name, dtype: int64

In [53]:
# Count number of categories by conservation_status
categories_count_by_status = df_species_info_group.groupby('conservation_status')['category'].nunique()
categories_count_by_status

conservation_status
Endangered            5
In Recovery           1
Species of Concern    7
Threatened            4
Name: category, dtype: int64